In [ ]:
# Common libs to import everywhere
import gc
import numba
import numpy as np


# Loading data
- reading one-hot encoded data, generated by the notebook AggLogistic-EncodeData.ipynb

In [ ]:
name = f"../data/encodedAggData_C_D_S_X_Xbis_n_pairs.npz"


In [ ]:
data = np.load(name)
C = data["C"]
D = data["D"]
S = data["S"]
X_test = data["X_test"]
X_another_set = data["X_another_set"]
nb_samples_agg = data["nb_samples_agg"]
pairs = data["pairs"]


In [ ]:
# Labels  (Used for computing validation score, not used for training!)

Y_clicks_test = np.loadtxt("../data/y_test.csv", skiprows=1, delimiter=",", dtype=np.int32, usecols=(0,))
Y_sales_test = np.loadtxt("../data/y_test.csv", skiprows=1, delimiter=",", dtype=np.int32, usecols=(1,))

Y_clicks_another_set = np.loadtxt(
    "../data/criteo-ppml-challenge-adkdd21-dataset-additional-test-data.csv",
    skiprows=1,
    delimiter=",",
    dtype=np.int32,
    usecols=(19,),
)
Y_sales_another_set = np.loadtxt(
    "../data/criteo-ppml-challenge-adkdd21-dataset-additional-test-data.csv",
    skiprows=1,
    delimiter=",",
    dtype=np.int32,
    usecols=(20,),
)


In [ ]:
run_on_sales = True
verbose = False
logfile = f"results_agglogistic_{'sales' if run_on_sales else 'clicks'}.log"
logfile


# Training code
- logistic regresion model sigmoid( w.x + b) 

In [ ]:
def print_and_log(x):
    print(x)
    with open(logfile, "a+") as handle:
        handle.write(x + "\n")

In [ ]:
def logit(x):
    return -np.log((1 / x) - 1)


@numba.njit
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


@numba.njit
def predict(w, bias, x):
    # bias = -2.2  ##  hardcoded bias, ~=  logit( nbclick/ nb displays  )
    return sigmoid(w[x].sum() + bias)


@numba.njit
def predicts(w, bias, xs):
    return np.array([predict(w, bias, x) for x in xs])


@numba.njit
def project(X, y, n):
    p = np.zeros(n)
    len_y = len(y)
    for i in range(0, len_y):
        xi = X[i]
        yi = y[i]
        for h in xi:
            p[h] += yi
    return p


def LLH(prediction, y):
    llh = np.log(prediction) * y + np.log(1 - prediction) * (1 - y)
    return sum(llh) / len(y)


def Entropy(y):
    py = sum(y > 0) / len(y)
    return py * np.log(py) + (1 - py) * np.log(1 - py)


def NLlh_(prediction, y):
    if any(prediction <= 0) or any(prediction >= 1):
        return np.nan
    h = Entropy(y)
    llh = LLH(prediction, y)
    return (h - llh) / h


def Nllh(w, bias, X, y):
    preds = predicts(w, bias, X)
    return NLlh_(preds, y)


l2 = 500


def gradient(w, pC, C, l2, D=None, count_agg_X=None):

    pc_corrected = pC
    c_corrected = C

    if count_agg_X is not None:
        # rescaling clicks or predicted clicks by the ratio between displays in aggregated data
        # and displays in (re-aggregated) granular dataset X
        pc_corrected = pC * np.minimum(D / (count_agg_X + 1), 1)
        c_corrected = C * np.minimum(count_agg_X / (D + 1), 1)

    g = pc_corrected - c_corrected

    g += l2 * w  ## regularization term 1/2 * l2 * sum(w*w)
    g[0] = 0  ##  coordinate 0 is 'missing' modality, shared between all pairs. not trying to learn it.

    return g


# Gradient rescaled by inverse diagonal Hessian
def descent_direction(w, bias, X, C, global_scale_factor, D, count_agg_X, l2):
    preds = predicts(w, bias, X)
    pC = project(X, preds, len(w)) * global_scale_factor
    g = gradient(w, pC, C, l2, D, count_agg_X)
    h = pC + l2 
    dir_ = -g / h
    return dir_


verbose = False


def train(
    bias,
    X,
    C,
    scale_factor,
    D,
    count_agg_X,
    l2,
    alpha=0.01,
    maxiters=100,
    X_validation=None,
    Y_validation=None,  # to monitor progress during learning
    w=None,
):
    ## Init model
    w = np.zeros(len(C)) if w is None else w
    nbiters = 0
    for i in range(0, maxiters):
        nbiters += 1
        direction = descent_direction(w, bias, X, C, scale_factor, D, count_agg_X, l2)
        w += alpha * direction
        if verbose and X_validation is not None:
            nllh = Nllh(w, bias, X_validation, Y_validation)
            print(f"{nbiters}  --  {nllh:.4f}    ", end="\r")
            if 9 == i % 10:
                print("")
    return w, bias


In [ ]:
## Compiling numba
n = len(D)
w = np.zeros(n)

preds = predicts(w, -2.2, X_test[:10, :])
p = project(X_test[:10, :], preds, n)


In [ ]:
def run(
    D,
    C,
    nb_samples_agg,  # aggregated data
    gaussian_sigma,  # Stdev of noise which should be added to agg data
    X_train,
    X_validation,
    Y_validation,  # to monitor training
    l2,  # L2 regularization weight
    use_by_coordinate_rescaling,
    min_displays=None,  # remove crossfeatures with less than minDisplays
    alpha=0.01,
    maxiters=100,  # optimizer stepsize and nb iters
    w=None,
):

    nb_samples_not_agg = len(X_train)
    scale_factor = nb_samples_agg / nb_samples_not_agg
    # print("scaleFactor", scaleFactor)

    count_agg_X = None
    if use_by_coordinate_rescaling:
        # aggregated number of displays; computed on X_test, rescaled to size of agg dataset.
        count_agg_X = project(X_train, np.ones(X_train.shape[0]), n) * scale_factor

    D = D.copy()
    C = C.copy()

    ## Adding noise where there is data ( Not truly diff private, but similar to the challenge)
    if gaussian_sigma > 0:
        D += np.random.normal(0, gaussian_sigma, len(D))
        C += np.random.normal(0, gaussian_sigma, len(D))

        D[0] = 0  #  "other" modality
        C[0] = 0

    if min_displays is not None:
        ind = np.where(D < min_displays)[0]
        D[ind] = 0
        C[ind] = 0
        # print( "Proportion of crosses set to 0"  ,  len(ind) / len(D) )

    ## removing absurd values (which might be observed because of the noise)
    D = np.maximum(D, 0)
    C = np.maximum(C, 0)
    C = np.minimum(C, D)

    bias = logit(C.sum() / D.sum())  # hardcode bias, I did not implement the gradient on the bias

    # print( "sigma",gaussianSigma,  "l2",l2 , "minDisplays", minDisplays,
    #       "scaleFactor",scaleFactor ,
    #      "useRescaling", CountAggX is not None, "noiseModelScaling",noiseModelScaling  )
    w, bias = train(
        bias,
        X_train,
        C,
        scale_factor,
        D,
        count_agg_X,
        l2=l2,
        alpha=alpha,
        maxiters=maxiters,
        X_validation=X_validation,
        Y_validation=Y_validation,
        w=w,
    )
    return w, bias


In [ ]:
def run_(
    nb_train_samples,
    train_with_X_test=False,
    gaussian_sigma=0,  # noise level
    l2=100,  # L2 regularization weight
    use_by_coordinate_rescaling=True,
    alpha=0.01,
    maxiters=100,
):

    if not run_on_sales:  ## Global param
        agg_labels = C
        Y_test = Y_clicks_test
        Y_another_set = Y_clicks_another_set

    else:
        agg_labels = S
        Y_test = Y_sales_test
        Y_another_set = Y_sales_another_set

    X_validation = X_another_set[-1_000_000:, :]
    Y_validation = Y_another_set[-1_000_000:]

    X_train = X_test if train_with_X_test else X_another_set
    ind = np.random.permutation(X_train.shape[0])[:nb_train_samples]
    X_train = X_train[ind, :]

    w, bias = run(
        D,
        agg_labels,
        nb_samples_agg,  # aggregated data
        gaussian_sigma,  # noise level
        X_train,  # note that Ytrain is only used to compute score during training. Works with Ytrain = None
        X_validation,
        Y_validation,  # to monitor training && pick parameters
        l2=l2,  # L2 regularization weight
        use_by_coordinate_rescaling=use_by_coordinate_rescaling,
        min_displays=None,  # remove crossfeatures with less than minDisplays
        alpha=alpha,
        maxiters=maxiters,
    )

    llh_valid = Nllh(w, bias, X_validation, Y_validation)
    llh_test = Nllh(w, bias, X_test, Y_test)

    tolog = (
        f"run_on_sales:{run_on_sales}; llh_valid:{llh_valid};llh_test:{llh_test}; nb_train_samples:{nb_train_samples};"
        + f"gaussian_sigma:{gaussian_sigma};train_with_X_test:{train_with_X_test};"
        + f"l2:{l2};use_by_coordinate_rescaling:{use_by_coordinate_rescaling};"
        + f"alpha:{alpha};maxiters:{maxiters}"
    )
    print_and_log(tolog)

    return w, bias


# Running experiments

In [ ]:
nb_train_samples = len(X_test)
print(nb_train_samples)


In [ ]:
default_l2 = 1_000
w, bias = run_(
    nb_train_samples,
    train_with_X_test=True,
    gaussian_sigma=0,  # noise level
    l2=default_l2,  # L2 regularization weight
    use_by_coordinate_rescaling=True,
    alpha=0.01,
    maxiters=100,
)


In [ ]:
print_and_log("Benching L2")
for l2 in [25, 50, 100, 200, 400, 800, 1000, 2000, 4000, 8000, 16000]:
    w, bias = run_(
        nb_train_samples,
        train_with_X_test=True,
        gaussian_sigma=0,  # noise level
        l2=l2,  # L2 regularization weight
        use_by_coordinate_rescaling=True,
        alpha=0.01,
        maxiters=100,
    )


In [ ]:
print_and_log("Benching L2 with train_with_X_test = False")
for l2 in [25, 50, 100, 200, 400, 800, 1000, 2000, 4000, 8000, 16000]:
    w, bias = run_(
        nb_train_samples,
        train_with_X_test=False,
        gaussian_sigma=0,  # noise level
        l2=l2,  # L2 regularization weight
        use_by_coordinate_rescaling=True,
        alpha=0.01,
        maxiters=100,
    )


In [ ]:
print_and_log("Benching alpha")

for alpha in [0.01, 0.05, 0.01 / 5]:
    w, bias = run_(
        nb_train_samples,
        train_with_X_test=True,
        gaussian_sigma=0,  # noise level
        l2=default_l2,  # L2 regularization weight
        use_by_coordinate_rescaling=True,
        alpha=alpha,
        maxiters=100,
    )


In [ ]:
print_and_log("Benching maxiter")

for maxiters in [20, 50, 100, 200]:
    w, bias = run_(
        nb_train_samples,
        train_with_X_test=True,
        gaussian_sigma=0,  # noise level
        l2=default_l2,  # L2 regularization weight
        use_by_coordinate_rescaling=True,
        alpha=0.01,
        maxiters=maxiters,
    )


In [ ]:
## no rescaling
print_and_log("No rescaling, benching l2")

for l2 in [25, 50, 100, 200, 400, 800, 1000, 2000, 4000, 8000, 16000]:
    w, bias = run_(
        nb_train_samples,
        train_with_X_test=True,
        gaussian_sigma=0,  # noise level
        l2=l2,  # L2 regularization weight
        use_by_coordinate_rescaling=False,
        alpha=0.01,
        maxiters=100,
    )


In [ ]:
## trainWithXTest = False, changing nbsamples
print_and_log("trainWithXTest = False, changing nb_train_samples")

for nb_samples in [10_000, 20_000, 50_000, 100_000, 200_000, 500_000, 1_000_000, 2_000_000, X_another_set.shape[0]]:
    for i in range(0, 5):
        gc.collect()
        w, bias = run_(
            nb_samples,
            train_with_X_test=False,
            gaussian_sigma=0,  # noise level
            l2=default_l2,  # L2 regularization weight
            use_by_coordinate_rescaling=True,
            alpha=0.01,
            maxiters=100,
        )


In [ ]:
## With noise; and quick l2 bench
print_and_log("With noise")

for sigma in [10, 17, 50, 250, 1_000, 5_000, 25_000, 100_000]:
    for l2 in [1000, 4000, 16_000, 64_000]:
        for i in range(0, 5):
            gc.collect()
            w, bias = run_(
                nb_train_samples,
                train_with_X_test=True,
                gaussian_sigma=sigma,  # noise level
                l2=default_l2,  # L2 regularization weight
                use_by_coordinate_rescaling=True,
                alpha=0.01,
                maxiters=100,
            )
